In [4]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from torch.utils.data import random_split,DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import gc
import torchmetrics

In [5]:
class Config:
    IMAGE_SIZE   = (224,224)
    TRAIN_FOLDER = './cat-dog/training_set/'
    TEST_FOLDER  = './cat-dog/test_set/'
    DEVICE       = "cuda:0" if torch.cuda.is_available() else "cpu"
    SPLIT        = (0.7,0.3)
    TRANSFORMS   =  transforms.Compose([transforms.Resize(size=IMAGE_SIZE),
                                       transforms.ToTensor()])
    BATCH_SIZE   = 5
    MODEL_DIR    = 'Pytorch/predefined/'
    EPOCH        =  10


    torch.hub.set_dir(MODEL_DIR)    

In [38]:
class TrainTestLoader:
    def __init__(self):
        """ DataLoader Class """
        pass
    def get_data(self, data_folder, split_ratio=Config.SPLIT,
                 split=True, batch_size=Config.BATCH_SIZE,
                 transforms=Config.TRANSFORMS):
        if split:
            """ Call method for quick data loading """
            self.train = ImageFolder(data_folder,transform=transforms)
            self.batch_size = batch_size
            self.split_ratio = split_ratio
            tr,ts = int(len(self.train)*split_ratio[0]),len(self.train)-int(len(self.train)*split_ratio[0])
            train_ds,val_ds = random_split(self.train, [tr,ts])
            self.train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
            self.val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
            return self.train_loader,self.val_loader
        else:
            self.test = ImageFolder(data_folder,transform=transforms)
            self.test_loader = DataLoader(self.test)
            return self.test_loader
dl = TrainTestLoader()
train_loader,val_loader = dl.get_data(Config.TRAIN_FOLDER)
test_loader = dl.get_data(Config.TEST_FOLDER,split=False)

In [40]:
len(train_loader),len(val_loader)

(1121, 481)

# Loading and Modifying Resnet

In [7]:
gc.collect()
resnet    = torch.hub.load('pytorch/vision:v0.10.0','resnet18', pretrained=False)
num_ftrs  = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs,1)
resnet    = resnet.to('cuda')

Using cache found in Pytorch/predefined/pytorch_vision_v0.10.0


In [19]:
target = torch.tensor([1,0])
preds  = torch.tensor([0.6,0.1])
accuracy = torchmetrics.Accuracy()
accuracy(preds, target)

tensor(1.)

# Model Metrics

In [41]:
class Metrics:
    def score(yhat,y):
        """Accuracy Score

        Args:
            yhat (float): probabilities
            y (integer(0/1)): class label 

        Returns:
            float: range 0-1
        """
        accuracy = torchmetrics.Accuracy()
        return accuracy(yhat,y)
    def f1(yhat,y):
        """F1-Score

        Args:
            yhat (float): probabilities
            y (integer(1/0)): class label

        Returns:
            float: range 0-1
        """
        f1 = torchmetrics.F1Score()
        return f1(yhat,y)
    

In [67]:
target  = torch.tensor([0,1])
preds = torch.tensor([0.1,0.9])
Metrics.score(preds,target)

tensor(1.)

# Attaching Model

In [118]:
class ModelFinal(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def make(self, resnet):
        self.resnet = resnet
    def train(self, X_cuda, y_cuda):
        out = self(X_cuda)
        return torch.nn.functional.binary_cross_entropy(out, y_cuda)
    def fit(self, train_loader, val_loader=None, epochs=1, lr=0.001):
        opt = torch.optim.Adam(self.parameters(),lr=lr)
        for epoch in range(epochs):
            total_avg_loss = 0
            total_loss = 0
            iterator_loader = tqdm.tqdm(train_loader, desc='Train Batch', total=len(train_loader))
            for iteration,batch in enumerate(iterator_loader):
                X_cuda = batch[0].to(dtype=torch.float32, device=Config.DEVICE)
                y_cuda = batch[1].to(dtype=torch.float32, device=Config.DEVICE).unsqueeze(1)
                loss = self.train(X_cuda, y_cuda)
                total_loss += loss.item()
                total_avg_loss = total_loss/(iteration+1)
                loss.backward()
                opt.step()
                opt.zero_grad()
                torch.cuda.empty_cache()
                iterator_loader.set_postfix({'total batch loss':loss.item(), 'total avg loss': total_avg_loss})
            print()
            self.validate(val_loader)
            gc.collect()

    def validate(self,val_loader):
        average_val_score = 0
        average_f1_score = 0
        total_f1 = 0
        total_score = 0
        iterator_loader = tqdm.tqdm(val_loader, desc='Val Batch', total=len(val_loader))
        for iteration,batch in enumerate(iterator_loader):
            X_val_cuda = batch[0].to(dtype=torch.float32, device=Config.DEVICE)
            y_val = torch.unsqueeze(batch[1].to(dtype=torch.int32), dim=1)
            predictions  = self.predict(X_val_cuda).to('cpu')
            
            accuracy = Metrics.score(predictions, y_val)
            total_score += accuracy.item()
            average_val_score  = total_score/(iteration+1)
            
            f1 = Metrics.f1(predictions,y_val)
            total_f1 += f1.item()
            average_f1_score = total_f1/(iteration+1)
            
            iterator_loader.set_postfix({'average f1 score':average_f1_score, 'average accuracy score': average_val_score})
            
        return average_val_score, average_f1_score
            
    def forward(self, X):
        model_output = self.resnet(X)
        softmax_output = torch.nn.Sigmoid()(model_output)
        return softmax_output
    def predict(self,X):
        with torch.no_grad():
            return self(X)

In [93]:
val = torch.tensor([0])
int(val[0])

0

In [79]:
# itr = tqdm.tqdm([1,2,3,4,5,6,7],desc='hello')
# for i,batch in enumerate(itr):
#     print(i)
#     itr.set_postfix({'i':i})

In [119]:
del model_attached
gc.collect()

404

In [120]:
model_attached = ModelFinal()
model_attached.make(resnet=resnet)

In [121]:
for x,y in val_loader:
    break

In [122]:
x.shape,y.shape

(torch.Size([5, 3, 224, 224]), torch.Size([5]))

In [123]:
torch.unsqueeze(y,dim=1).shape

torch.Size([5, 1])

In [124]:
model_attached.validate(val_loader)

Val Batch: 100%|█| 481/481 [00:17<00:00, 27.93it/s, average f1 score=0.513, average accuracy score=0.593


(0.5931393058347108, 0.5129739698526021)

In [109]:
model_attached(x.to(dtype=torch.float32,device=Config.DEVICE)).shape

torch.Size([5, 1])

In [104]:
model_attached.validate(val_loader)

Val Batch:   0%|                                                                | 0/481 [00:00<?, ?it/s]


ValueError: The highest label in `target` should be smaller than the size of the `C` dimension of `preds`.

In [103]:
model_attached.fit(train_loader=train_loader,val_loader=val_loader,epochs=2)

Train Batch: 100%|████| 1121/1121 [01:55<00:00,  9.73it/s, total batch loss=0.703, total avg loss=0.677]

TypeError: object of type 'NoneType' has no len()

In [54]:
for x,y in val_loader:
    break

In [55]:
x.shape

torch.Size([5, 3, 224, 224])

In [56]:
x.device

device(type='cpu')

In [57]:
y_pred = model_attached.predict(x.to(device=Config.DEVICE))

In [58]:
y_pred.shape

torch.Size([5, 1])

In [59]:
y_pred

tensor([[0.4551],
        [0.4991],
        [0.4903],
        [0.5212],
        [0.5168]], device='cuda:0')